# SAE-76 - TF-IDF Optimisé

**En tant que** ML engineer  
**Je veux** optimiser les paramètres TF-IDF  
**Afin de** améliorer la qualité de la représentation

## Objectifs

- Explorer les paramètres (max_features, min_df, max_df, ngrams)
- Comparer différentes configurations
- Sélectionner la meilleure configuration
- Sauvegarder le modèle optimal

## Input/Output

- **Input**: `data/cleaned/reviews_text_cleaned.parquet`
- **Output**: `outputs/models/tfidf_vectorizer.pkl`

In [ ]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
import time

pd.set_option('display.max_columns', None)
print("✅ Imports réussis")

## 1. Chargement des Données

In [ ]:
# Configuration des chemins
DATA_PATH = Path('../../data/cleaned/reviews_text_cleaned.parquet')
MODEL_PATH = Path('../../outputs/models/tfidf_vectorizer.pkl')

# Créer le dossier output si nécessaire
MODEL_PATH.parent.mkdir(parents=True, exist_ok=True)

print(f"📂 Chargement: {DATA_PATH}")

In [ ]:
# Chargement des données
df = pd.read_parquet(DATA_PATH)
print(f"✅ Données chargées: {len(df):,} reviews")

# Vérification de la colonne text_cleaned
if 'text_cleaned' not in df.columns:
    raise KeyError("❌ La colonne 'text_cleaned' est manquante!")

# Remplir les NaNs potentiels
df['text_cleaned'] = df['text_cleaned'].fillna('')

# Aperçu
df[['text_cleaned']].head()

## 2. Exploration et Comparaison des Configurations

Nous allons tester 3 configurations:
1. **Default**: Paramètres par défaut
2. **Limited**: Vocabulaire limité (max_features=5000, min_df=5, max_df=0.8)
3. **Bigrams**: Unigrams + Bigrams avec vocabulaire limité

In [ ]:
# Définition des vectorizers
vectorizers = {
    "Default": TfidfVectorizer(),
    "Limited": TfidfVectorizer(
        max_features=5000,
        min_df=5,
        max_df=0.8
    ),
    "Bigrams": TfidfVectorizer(
        max_features=5000,
        ngram_range=(1, 2),
        min_df=5
    )
}

results = []

print("🚀 Début de la comparaison...")

for name, vect in vectorizers.items():
    print(f"\n⏳ Traitement config: {name}...")
    start_time = time.time()
    
    # Fit transform
    X = vect.fit_transform(df['text_cleaned'])
    
    # Calculs métriques
    sparsity = (1 - X.nnz / (X.shape[0] * X.shape[1])) * 100
    vocab_size = len(vect.get_feature_names_out())
    memory_mb = X.data.nbytes / (1024 * 1024)
    duration = time.time() - start_time
    
    print(f"   ✅ Terminé en {duration:.2f}s")
    print(f"   📊 Shape: {X.shape}")
    print(f"   📚 Vocab size: {vocab_size:,}")
    print(f"   🕸️ Sparsity: {sparsity:.4f}%")
    
    results.append({
        'Configuration': name,
        'Vocab Size': vocab_size,
        'Sparsity (%)': sparsity,
        'Memory (MB)': memory_mb,
        'Time (s)': duration
    })

In [ ]:
# Afficher le tableau comparatif
results_df = pd.DataFrame(results)
print("\n🏆 Résultats Comparatifs:")
display(results_df)

## 3. Analyse et Choix

### Analyse des résultats
- **Default**: Vocabulaire très large, risque de bruit et d'overfitting. Lent.
- **Limited**: Vocabulaire contrôlé, plus rapide, focus sur les mots fréquents.
- **Bigrams**: Capture des contextes (ex: "not good"), vocabulaire contrôlé par max_features.

### Choix Final
Nous choisissons la configuration **Bigrams** car elle permet de capturer plus de sens (négation, superlatifs composés) tout en gardant une dimensionnalité gérable (5000 features).

## 4. Sauvegarde du Meilleur Modèle

In [ ]:
best_model_name = "Bigrams"
best_vectorizer = vectorizers[best_model_name]

print(f"💾 Sauvegarde du modèle '{best_model_name}' vers {MODEL_PATH}...")

with open(MODEL_PATH, 'wb') as f:
    pickle.dump(best_vectorizer, f)

print("✅ Modèle sauvegardé avec succès!")

In [ ]:
# Vérification du chargement
with open(MODEL_PATH, 'rb') as f:
    loaded_vect = pickle.load(f)

print(f"🔍 Vérification rechargement: {type(loaded_vect)}")
print(f"   Params: ngram_range={loaded_vect.ngram_range}, max_features={loaded_vect.max_features}")